In [13]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import json
from gensim.utils import deaccent

unable to import 'smart_open.gcs', disabling that module


In [21]:
class Downloader():
    def __init__(self):
        self.link = 'https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha?_token=pr1lf-vKwDFfmFbICiz2PfC-Zdwq-2JolXi4MeMHsrw&page=1'
        self.request = requests.get(self.link)
        self.request.encoding='UTF-8'
        self.soup = BeautifulSoup(self.request.text,'lxml')
        
    def get_data(self):
        descrips = []
        values = []
        vals = self.soup.findAll('strong', {'class':'product__value'})
        for vl in vals:
            values.append(vl.text.strip())
        img = self.soup.findAll('img')
        for i in img:
            if 'Pronájem' and 'obr. č. 1' in i['alt']:
                    info = i['alt'].split(',')[0:4]
                    descrips.append(info)
        count = 0
        for pp in values:
            descrips[count].append(pp)
            descrips[count][0] = descrips[count][0][-4:].strip()
            count += 1
        for item in descrips:
            prices = item.pop(4).split('+')
            item.append(re.sub("[^0-9]", "", prices[0]))
            item.append(re.sub("[^0-9]", "", prices[1]))
        dicts = {}
        count = 0
        for item in descrips:
            dict = {}
            dict['Size'] = item[0]
            dict['m2'] = re.sub("[^0-9]", "", item[1])
            dict['Street'] = deaccent(item[2])
            dict['District'] = deaccent(item[3])
            dict['Base Price'] = int(item[4])
            dict['Utilities Price'] = int(item[5])
            dict['Total Price'] = int(item[4]) + int(item[5])
            dicts[count] = dict
            count += 1
        data = json.dumps(dicts)
        with open('bezrealitky.json', 'w') as F:
            F.write(data)
        
        
            
        

In [22]:
a = Downloader()
a.get_data()